# Entraînement d'un réseau de neurones pour jouer au Go

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/auduvignac/deep_learning_go/blob/main/src/train_go_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

## Description

- [https://www.lamsade.dauphine.fr/~cazenave/DeepLearningProject.html](https://www.lamsade.dauphine.fr/~cazenave/DeepLearningProject.html)  
- L'objectif est d'entraîner un réseau pour jouer au jeu de Go.  
- Afin de garantir une équité en termes de ressources d'entraînement, le nombre de paramètres des réseaux soumis doit être inférieur à 100 000.  
- Le nombre maximal d'étudiants par équipe est de deux.  
- Les données utilisées pour l'entraînement proviennent des parties auto-jouées du programme Katago Go.  
- Le jeu de données d'entraînement contient un total de 1 000 000 de parties différentes.  
- Les données d'entrée sont composées de 31 plans de taille 19x19 :  
  - Couleur au trait  
  - Échelles  
  - État actuel sur deux plans  
  - Deux états précédents sur plusieurs plans  
- Les cibles de sortie sont :  
  - **La politique** : un vecteur de taille 361 avec `1.0` pour le coup joué, `0.0` pour les autres coups.  
  - **La valeur** : une valeur entre `0.0` et `1.0` fournie par la recherche d'arbre Monte-Carlo, représentant la probabilité de victoire de Blanc.

- Le projet a été écrit et fonctionne sous Ubuntu 22.04.  
- Il utilise TensorFlow 2.9 et Keras pour le réseau.  
- Un exemple de réseau convolutionnel avec deux têtes est donné dans le fichier `golois.py` et est sauvegardé dans le fichier `test.h5`.  
- Les réseaux que vous concevez et entraînez doivent également avoir les mêmes têtes de politique et de valeur et être sauvegardés au format `.h5`.  
- Un exemple de réseau et un épisode d'entraînement sont fournis dans le fichier `golois.py`.  
- Si vous souhaitez compiler la bibliothèque Golois, vous devez installer **Pybind11** et exécuter `compile.sh`.

## Tournois

- Toutes les deux semaines environ, un tournoi est organisé entre les réseaux téléchargés.  
- Chaque nom de réseau correspond aux noms des étudiants qui ont conçu et entraîné le réseau.  
- Le modèle doit être sauvegardé au format **Keras h5**.  
- Un tournoi en **round robin** sera organisé et les résultats seront envoyés par e-mail.  
- Chaque réseau sera utilisé par un moteur **PUCT**, qui disposera de **2 secondes de temps CPU** par coup pour jouer dans le tournoi.

## Exemple de réseau

```python
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import gc
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

import golois

planes = 31
moves = 361
N = 10000
epochs = 20
batch = 128
filters = 32

input_data = np.random.randint(2, size=(N, 19, 19, planes))
input_data = input_data.astype("float32")

policy = np.random.randint(moves, size=(N,))
policy = keras.utils.to_categorical(policy)

value = np.random.randint(2, size=(N,))
value = value.astype("float32")

end = np.random.randint(2, size=(N, 19, 19, 2))
end = end.astype("float32")

groups = np.zeros((N, 19, 19, 1))
groups = groups.astype("float32")

print("Tensorflow version", tf.__version__)
print("getValidation", flush=True)
golois.getValidation(input_data, policy, value, end)


input = keras.Input(shape=(19, 19, planes), name="board")
x = layers.Conv2D(filters, 1, activation="relu", padding="same")(input)
for i in range(5):
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
policy_head = layers.Conv2D(
    1,
    1,
    activation="relu",
    padding="same",
    use_bias=False,
    kernel_regularizer=regularizers.l2(0.0001),
)(x)
policy_head = layers.Flatten()(policy_head)
policy_head = layers.Activation("softmax", name="policy")(policy_head)
value_head = layers.Conv2D(
    1,
    1,
    activation="relu",
    padding="same",
    use_bias=False,
    kernel_regularizer=regularizers.l2(0.0001),
)(x)
value_head = layers.Flatten()(value_head)
value_head = layers.Dense(
    50, activation="relu", kernel_regularizer=regularizers.l2(0.0001)
)(value_head)
value_head = layers.Dense(
    1,
    activation="sigmoid",
    name="value",
    kernel_regularizer=regularizers.l2(0.0001),
)(value_head)

model = keras.Model(inputs=input, outputs=[policy_head, value_head])

model.summary()

model.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.0005, momentum=0.9),
    loss={
        "policy": "categorical_crossentropy",
        "value": "binary_crossentropy",
    },
    loss_weights={"policy": 1.0, "value": 1.0},
    metrics={"policy": "categorical_accuracy", "value": "mse"},
)

for i in range(1, epochs + 1):
    print("epoch " + str(i))
    golois.getBatch(input_data, policy, value, end, groups, i * N)
    history = model.fit(
        input_data,
        {"policy": policy, "value": value},
        epochs=1,
        batch_size=batch,
    )
    if i % 5 == 0:
        gc.collect()
    if i % 20 == 0:
        golois.getValidation(input_data, policy, value, end)
        val = model.evaluate(
            input_data, [policy, value], verbose=0, batch_size=batch
        )
        print("val =", val)
        model.save("test.h5")
```

## Instructions :  
- Entraînez un réseau pour jouer au Go.  
- Soumettez les réseaux entraînés **avant samedi soir**.  
- Tournoi des réseaux **chaque dimanche**.  
- Téléchargez un réseau **avant la fin de la session**.

## Objectif du projet

Ce projet a pour objectif d’implémenter et d’évaluer plusieurs architectures de réseaux de neurones convolutionnels appliquées à la modélisation du jeu de Go. Les architectures ciblées sont les suivantes :

- **ResNet** : réseaux résiduels profonds facilitant l’apprentissage de modèles très profonds grâce aux connexions de saut (skip connections).

- **MobileNet** : architectures légères conçues pour les environnements contraints, utilisant des convolutions séparables en profondeur (depthwise separable convolutions) pour réduire le nombre de paramètres.

- **ConvNeXt** : réseaux convolutionnels modernes inspirés des Transformers, conçus comme une évolution des CNN classiques avec des performances compétitives sur ImageNet.

- **ShuffleNet** : modèles optimisés pour l'efficacité computationnelle, combinant group convolutions et opérations de réorganisation (channel shuffle) pour limiter le coût en calcul tout en maintenant de bonnes performances.

Ces modèles seront adaptés, entraînés et comparés dans le cadre d’un apprentissage supervisé pour prédire les coups dans des parties de Go.

## Mise en place de l'environnement de travail

Installation préalable de l’API de Go et de la bibliothèque Golois.

In [ ]:
!wget https://www.lamsade.dauphine.fr/~cazenave/project2025.zip
!unzip project2025.zip
!ls -l

Ensuite, nous allons installer les dépendances nécessaires à l'entraînement du réseau de neurones.

In [ ]:
%pip uninstall -y tensorflow
%pip install tensorrt-bindings==8.6.1
%pip install --extra-index-url https://pypi.nvidia.com tensorrt-libs
%pip install tensorflow[and-cuda]==2.15.0

**Remarque importante :** Cette étape réalisée, il est nécessaire de redémmarer la session par l'intermédiaire de l'onglet « Exécution » et « Redémarrer le session ».

# Choix méthodologique : Focalisation sur la profondeur des réseaux

Dans le cadre de cette étude, le choix a été fait de **faire varier exclusivement la profondeur des architectures** (nombre de blocs) tout en maintenant constantes les autres paramètres, notamment le **nombre de filtres**.

Cette stratégie s'inscrit dans une logique rigoureuse, à la croisée de contraintes pratiques et d'objectifs analytiques :

- **Objectif ciblé** : L'étude vise à évaluer l'**impact direct de la profondeur** sur la capacité d'un modèle à apprendre les régularités d'un jeu de stratégie complexe comme le Go, dans un **cadre contraint en ressources**.
- **Contrainte stricte de complexité** : Tous les modèles sont limités à un **maximum de 100 000 paramètres**, ce qui exclut toute montée en capacité par simple élargissement du réseau.
- **Contrôle expérimental** : En maintenant constant le nombre de filtres, on **isole l'effet de la profondeur** sur la performance, ce qui permet une analyse plus fine de son influence.
- **Approche progressive** : En explorant un grand nombre de profondeurs différentes (jusqu'à 28 blocs pour `MobileNet`), le travail permet de **cartographier la relation entre profondeur et performance** dans un budget de complexité fixe.
- **Justification temporelle** : Compte tenu des **contraintes de temps inhérentes au projet**, un balayage systématique de la profondeur a été privilégié à une exploration conjointe profondeur-largeur, qui aurait exigé davantage de ressources.

> Le choix de **ne faire évoluer que la profondeur des réseaux** dans une enveloppe de **paramètres limitée à 100 000** permet d'**évaluer précisément la complexité verticale** des architectures tout en **respectant un budget computationnel réaliste**. Ce cadre expérimental assure une **analyse structurée, reproductible et pertinente** dans le contexte du projet.


## Rappels Apprentissage par renforcement

« L'apprentissage par renforcement (en anglais, *reinforcement learning*, ou RL) est la branche de l'apprentissage automatique qui consiste à apprendre comment un agent doit se comporter dans un environnement de manière à maximiser une récompense. Naturellement, l’apprentissage par renforcement profond restreint la méthode d'apprentissage à l'apprentissage profond » ([Charniak, E. (2019). *Introduction au Deep Learning*. Dunod. p. 105](https://www.dunod.com/sciences-techniques/introduction-au-deep-learning)).

« Dans l'apprentissage par renforcement, un agent logiciel procède à des observations et réalise des actions au sein d'un environnement. En retour, il reçoit des récompenses. Son objectif est d'apprendre à agir de façon à maximiser les récompenses espérées sur le long terme » ([Géron, A. (2023). *Deep Learning avec Keras et TensorFlow* (3e éd.). O'Reilly. p. 440](https://www.dunod.com/sciences-techniques/deep-learning-avec-keras-et-tensorflow-mise-en-oeuvre-et-cas-concrets-0)).

« L'algorithme que l'agent logiciel utilise pour déterminer ses actions est appelé stratégie ou politique (*policy*). Cette politique peut être un réseau de neurones qui prend en entrée des observations et produit en sortie l'action à réaliser » ([Géron, A. (2023). *Deep Learning avec Keras et TensorFlow* (3e éd.). O'Reilly. p. 441](https://www.dunod.com/sciences-techniques/deep-learning-avec-keras-et-tensorflow-mise-en-oeuvre-et-cas-concrets-0)).


Dans le cas du jeu de go:
- L'agent est le programme qui joue au jeu ;
- L'environnement est le plateau de jeu ;
- Les récompenses sont les points gagnés ou perdus lors d'une partie ;
- La politique définit la manière dont l'agent choisit ses coups en fonction de l'état du plateau, dans le but de maximiser ses gains à long terme.


L'objectif de ce projet est de concevoir un réseau de neurones permettant de jouer au jeu de go.

## Importation des librairies

In [ ]:
import gc  # garbage collector
from abc import ABC, abstractmethod

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

from google.colab import files
from IPython.display import display
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.layers import (
    Input,
    Conv2D,
    DepthwiseConv2D,
    Dense,
    Concatenate,
    Add,
    ReLU,
    BatchNormalization,
    AvgPool2D,
    MaxPool2D,
    GlobalAveragePooling2D,
    Reshape,
    Permute,
    Lambda,
    Flatten,
    Activation,
)
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tqdm import tqdm

import golois

## Création de la classe abstraite `GONet`

La première étape consiste à implémenter une classe abstraite `GONet` dont l’objectif est de servir de **modèle générique** pour la construction de réseaux de neurones capables de prédire les coups joués (politique) et la probabilité de victoire (valeur) dans le cadre du jeu de Go. Elle fournit un cadre modulaire, réutilisable et traçable pour expérimenter les diverses architectures de CNN proposée : `ResNet`, `MobileNet`, `ConvNeXt`, `ShuffleNet`,  et garantit que toutes les variantes partagent une base commune et comparable, tant sur le plan technique (entrée, sortie, entraînement) que méthodologique (nombre de paramètres, métriques, etc.).

Cette classe regroupe l’ensemble des **comportements communs** à toutes les architectures : génération des données d’entrée et des cibles simulées, définition des entrées du modèle, création des têtes de sortie (politique et valeur), compilation, entraînement, évaluation, traçage des métriques et sauvegarde. L’enchaînement complet de ces étapes est centralisé dans la méthode `workflow()`.

Pour assurer une compréhension aisée de l'implémentation et se focaliser exclusivement sur la construction des quatre architectures proposées dans l'énoncé, les principes fondamentaux de la programmation orientée objet seront exploités :

- **Aabstraction** : via la méthode abstraite `set_backbone()`, qui impose aux classes dérivées de définir leur propre architecture du réseau (*backbone* convolutionnel) ;
- **Héritage** : les architectures spécifiques (`ResNet`, `MobileNet`, `ConvNeXt`, `ShuffleNet`) seront implémentées dans des classes filles (`ResNetGONet`, `MobileNetGONet`, `ConvNeXtGONet`, `ShuffleNetGONet`) qui hériteront de `GONet` et exploiteront l’ensemble des fonctionnalités définies dans la classe mère ;
- **Encapsulation** : les différentes étapes de préparation des données, de construction du modèle et d’entraînement sont regroupées dans des méthodes cohérentes, appelées par un point d’entrée unique (`workflow()`).

L'objectif est de **standardiser l’entraînement, l’évaluation et la comparaison** des modèles tout en respectant les contraintes du projet, notamment :

- Un maximum de **100 000 paramètres** ;
- Deux en-têtes de sortie fixes : **politique** (softmax) et **valeur** (sigmoïde) ;
- Données d’entrée : **31 plans 19×19** (représentation du plateau de Go) ;
- Données générées automatiquement pour simuler l'entraînement.

### Structure générale

Les méthodes implémentées dans la classe mère `GONet` sont résumées ci-dessous et décrites dans le paragraphe suivant.

```python
class GONet(ABC):
    # Méthodes utilitaires (initialisation et préparation des données)
    def __init__(...)
    def set_end(...)
    def set_groups(...)
    def set_input_data(...)
    def set_input_layer(...)
    def set_policy(...)
    def set_value(...)

    # Méthodes liées au modèle
    def create_policy_value_heads(...)
    def plot_model(...)
    def set_backbone(...) # méthode abstraite
    def set_model(...)

    # Méthodes d'entraînement et d’évaluation
    def log_accuracy(...)
    def plot_training_history(...)
    def train_model(...)
    def workflow(...)

    # Méthode de sauvegarde
    def save_model(...)
```

### Description des méthodes

#### Méthodes utilitaires (initialisation et préparation des données)

##### `__init__()`
- Initialise l’ensemble des composants nécessaires à la construction du modèle ;
- Génère des **données simulées aléatoires** pour les entrées (plans du plateau) et les cibles (politique, valeur, fin de partie, groupes) ;
- Prépare les structures internes pour l'entraînement et le suivi des métriques.

##### `set_end()` et `set_groups()`
- Produisent des représentations auxiliaires du plateau :
  - `end` : pour simuler des fins de partie ;
  - `groups` : pour simuler des regroupements de pierres.

##### `set_input_data()`
- Simule les **données d’entrée** sous la forme d’un tenseur `[N, 19, 19, 31]` représentant l’état du plateau à travers 31 plans.

##### `set_input_layer()`
- Définit la **couche d’entrée** du modèle : `Input(shape=(19, 19, 31))`.

##### `set_policy()`
- Génére des cibles pour la **tête de politique** sous forme de vecteurs one-hot de taille 361 (correspondant aux 361 positions du plateau 19x19).

##### `set_value()`
- Génére une **valeur binaire** (0 ou 1) représentant la probabilité de victoire de Blanc.

#### Méthodes liées au modèle

##### `create_policy_value_heads()`
- Ajoute les deux **têtes de prédiction** finales :
  - **Politique** : `Conv2D(1x1)` → `Flatten` → `Softmax` (sortie de taille 361) ;
  - **Valeur** : `Conv2D(1x1)` → `Dense(50)` → `Dense(1, Sigmoid)`.

- Compile le modèle avec :
  - Optimiseur : `SGD` avec momentum `0.9` ;
  - Pertes : `categorical_crossentropy` (politique), `binary_crossentropy` (valeur) ;
  - Métriques : `categorical_accuracy` et `MSE`.

- Vérifie que le nombre total de **paramètres** est inférieur à 100 000, sinon une exception est levée.

##### `plot_model()`
- Affiche la **structure graphique** du modèle Keras, incluant les couches et leurs dimensions.

##### `set_backbone()`
- Méthode **abstraite** (définie avec `@abstractmethod`) que chaque sous-classe doit redéfinir pour spécifier l’architecture du backbone convolutionnel.
- Doit produire un tenseur de forme `(None, 19, 19, 32)` affecté à `self.x`.

##### `set_model()`
- Assemble le modèle complet en combinant le **backbone** défini par la sous-classe et les têtes (politique et valeur).

#### Méthodes d'entraînement et d’évaluation

##### `log_accuracy(results_dict)`
- Ajoute la dernière valeur de **précision de la tête politique** dans un dictionnaire de résultats pour comparaison entre architectures.

##### `plot_training_history()`
- Affiche l’évolution de l’apprentissage sous forme de courbes :
  - Pertes : totale, politique, valeur ;
  - Métriques : précision politique (*categorical accuracy*) et MSE valeur.

##### `train_model()`
- Lance l’entraînement du modèle sur plusieurs *epochs* ;
- Enregistre les **pertes et métriques** à chaque époque ;
- Effectue une évaluation intermédiaire toutes les 20 *epochs* ;
- Libère la mémoire toutes les 5 *epochs* via `gc.collect()`.

##### `workflow(epochs, batch, name, log_accuracy_dict)`
- Lance le **pipeline complet** du modèle :
  1. Construction du backbone ;
  2. Création du modèle ;
  3. Entraînement ;
  4. Sauvegarde ;
  5. Affichage des courbes d’apprentissage ;
  6. Enregistrement de l’accuracy finale.

#### Méthode de sauvegarde

##### `save_model(name)`
- Sauvegarde le modèle Keras entraîné au format `.h5` dans le fichier spécifié (`name`), en vue d’une soumission ou d’une réutilisation ultérieure.

### Tableau récapitulatif des méthodes de `GONet`

| Méthode                   |  Description                                                | Catégorie                            |
|---------------------------|-------------------------------------------------------------|--------------------------------------|
| `__init__()`              | Initialise les composants et les données simulées           | Utilitaires / Initialisation         |
| `set_end()`               | Crée des fins de parties simulées                           | Utilitaires / Initialisation         |
| `set_groups()`            | Initialise les groupes de pierres                           | Utilitaires / Initialisation         |
| `set_input_data()`        | Génère les entrées simulées du plateau                      | Utilitaires / Initialisation         |
| `set_input_layer()`       | Définit la couche d’entrée du modèle                        | Utilitaires / Initialisation         |
| `set_policy()`            | Génère les cibles de la politique (one-hot)                 | Utilitaires / Initialisation         |
| `set_value()`             | Génère les cibles de la valeur (0 ou 1)                     | Utilitaires / Initialisation         |
| `create_policy_value_heads()` | Ajoute les têtes de sortie (politique, valeur) et compile le modèle | Modèle                               |
| `plot_model()`            | Affiche la structure du modèle                              | Modèle                               |
| `set_backbone()`          | Méthode abstraite pour définir l’architecture CNN           | Modèle (à implémenter dans les classes filles) |
| `set_model()`             | Assemble le backbone et les têtes                           | Modèle                               |
| `log_accuracy()`          | Enregistre la dernière accuracy dans un dictionnaire        | Entraînement / Évaluation            |
| `plot_training_history()` | Affiche les courbes de pertes et métriques                  | Entraînement / Évaluation            |
| `train_model()`           | Entraîne le modèle et collecte les métriques                | Entraînement / Évaluation            |
| `workflow()`              | Exécute le pipeline complet d’entraînement et d’évaluation  | Entraînement / Évaluation            |
| `save_model(name)`        | Sauvegarde le modèle au format `.h5`                        | Sauvegarde                           |


### Elements de précision sur l'interprétation des courbes de *loss* et des métriques

Les courbes d'entraînement illustrent l'évolution conjointe des erreurs (*losses*) et des performances (métriques) du modèle tout au long de l'apprentissage.
Ces valeurs sont collectées à chaque *epoch* au sein de la méthode `train_model`, qui enregistre les différentes mesures de *loss* et de précision pendant l'entraînement.

Le tableau ci-dessous synthétise leur signification :

#### Losses

| **Courbe**      | **Interprétation**                                                                 |
|-----------------|-------------------------------------------------------------------------------------|
| *Loss* totale | Somme pondérée des pertes de la tête policy et de la tête value.                   |
| *Policy Loss* | Perte de classification associée à la prédiction du coup (ex. : `categorical_crossentropy`). |
| *Value Loss*  | Perte de régression pour estimer l'issue de la partie (ex. : `mean_squared_error`). |

#### Métriques

| **Courbe**         | **Interprétation**                                                                 |
|--------------------|------------------------------------------------------------------------------------|
| *Policy Accuracy*| Taux de bonnes prédictions des coups joués, évalué sur la tête de policy.          |
| *Value MSE*      | Erreur quadratique moyenne sur la prédiction de la valeur de position (tête value).|


In [ ]:
class GONet(ABC):
    """
    Classe abstraite pour la construction d'un réseau de neurones pour le jeu de Go.

    Cette classe initialise des données simulées, définit les entrées et
    sorties du modèle, permet la construction des en-têtes de prédiction
    (politique et valeur), et gère l'entraînement et l'évaluation.

    Les sous-classes doivent obligatoirement implémenter `set_backbone()`,
    qui définit l'architecture du corps principal du réseau.

    La méthode `create_policy_value_heads` en charge de créer les en-têtes
    de sortie du modèle (politique et valeur) restera inchangée indépendamment
    de la structure du réseau de neurones utilisée.
    """

    # -------------------------------------
    # Méthodes utilitaires / initialisation
    # -------------------------------------

    def __init__(
        self,
        moves=361,
        N=10000,
        planes=31,
        max_params=100000,
        name="model",
    ):
        """
        Initialise les paramètres et génère les données d'entraînement aléatoires.

        Args:
            moves (int): Nombre total de coups possibles (361 pour un plateau 19x19).
            N (int): Nombre d'exemples dans le jeu de données.
            planes (int): Nombre de plans utilisés en entrée pour représenter le plateau.
        """
        self.moves = moves
        self.N = N
        self.planes = planes
        self.max_params = max_params
        self.nb_params = 0
        self.name = name
        self.set_input_data()
        self.set_policy()
        self.set_value()
        self.set_end()
        self.set_groups()
        self.set_input_layer()
        golois.getValidation(
            self.input_data, self.policy, self.value, self.end
        )
        self.loss_total = []
        self.policy_loss = []
        self.value_loss = []
        self.policy_acc = []
        self.value_mse = []

    def set_end(self):
        """
        Génère des représentations aléatoires de fin de partie.

        Args: None

        Returns:
            None: Affecte self.end (forme: [N, 19, 19, 2])
        """
        end = np.random.randint(2, size=(self.N, 19, 19, 2))
        self.end = end.astype("float32")

    def set_groups(self):
        """
        Initialise des groupes de pierres à zéro.

        Args: None

        Returns:
            None: Affecte self.groups (forme: [N, 19, 19, 1])
        """
        groups = np.zeros((self.N, 19, 19, 1))
        self.groups = groups.astype("float32")

    def set_input_data(self):
        """
        Génère les données d'entrée du réseau sous forme de tenseurs aléatoires.

        Args: None

        Returns:
            None: Affecte self.input_data (forme: [N, 19, 19, planes])
        """
        input_data = np.random.randint(2, size=(self.N, 19, 19, self.planes))
        self.input_data = input_data.astype("float32")

    def set_input_layer(self):
        """
        Définit la couche d'entrée du modèle.

        Args: None

        Returns:
            None: Affecte self.input_layer (couche d'entrée)
        """
        # Couche d'entrée : plateau de Go 19x19 avec P plans de caractéristiques
        self.input_layer = keras.Input(
            shape=(19, 19, self.planes), name="board"
        )

    def set_policy(self):
        """
        Génère des cibles de politique sous forme one-hot encodées.

        Args: None

        Returns:
            None: Affecte self.policy (forme: [N, moves])
        """
        policy = np.random.randint(self.moves, size=(self.N,))
        self.policy = keras.utils.to_categorical(policy)

    def set_value(self):
        """
        Génère des valeurs de victoire (0 ou 1) aléatoires.

        Args: None

        Returns:
            None: Affecte self.value (forme: [N])
        """
        value = np.random.randint(2, size=(self.N,))
        self.value = value.astype("float32")

    # -------------------------------------
    # Méthodes liées au modèle
    # -------------------------------------

    def create_policy_value_heads(self, verbose=False):
        """
        Crée les en-têtes de sortie du modèle : politique (softmax) et
        valeur (sigmoïde).

        Args:
            x (Tensor): Sortie du tronc du modèle.
            input_layer (Tensor): Couche d'entrée du modèle.

        Returns:
            keras.Model: Modèle compilé avec têtes de sortie et métriques
              définies.
        """
        try:
            expected_shape = (None, 19, 19, 32)

            for i, (a, e) in enumerate(zip(self.x.shape, expected_shape)):
                if e is not None and a != e:
                    raise ValueError(
                        f"""
                        - Forme inattendue pour self.x : {self.x.shape}.
                        - La dimension {i} vaut {a} au lieu de {e} ;
                        - Forme attendue complète : {expected_shape}
                      """
                    )
            # En-tête de politique
            policy_head = layers.Conv2D(
                1,
                1,
                activation="relu",
                padding="same",
                use_bias=False,
                kernel_regularizer=regularizers.l2(0.0001),
            )(self.x)
            policy_head = layers.Flatten()(policy_head)
            policy_head = layers.Activation("softmax", name="policy")(
                policy_head
            )
            # En-tête de valeur
            value_head = layers.Conv2D(
                1,
                1,
                activation="relu",
                padding="same",
                use_bias=False,
                kernel_regularizer=regularizers.l2(0.0001),
            )(self.x)
            value_head = layers.Flatten()(value_head)
            value_head = layers.Dense(
                50,
                activation="relu",
                kernel_regularizer=regularizers.l2(0.0001),
            )(value_head)
            value_head = layers.Dense(
                1,
                activation="sigmoid",
                name="value",
                kernel_regularizer=regularizers.l2(0.0001),
            )(value_head)

            model = keras.Model(
                inputs=self.input_layer,
                outputs=[policy_head, value_head],
                name=self.name,
            )

            if verbose:
                model.summary()

            # Vérification du nombre total de paramètres
            self.nb_params = model.count_params()
            if self.nb_params > self.max_params:
                raise Exception(
                    f"""
                  Le nombre de paramètres {self.nb_params} doit être inférieur à {self.max_params}
                """
                )
            else:
                if verbose:
                    print(
                        f"Le modèle contient {self.nb_params} paramètres, "
                        f"inférieur au seuil maximal ({self.max_params}). "
                        "Traitement poursuivi."
                    )

            model.compile(
                optimizer=keras.optimizers.SGD(
                    learning_rate=0.0005, momentum=0.9
                ),
                loss={
                    "policy": "categorical_crossentropy",
                    "value": "binary_crossentropy",
                },
                loss_weights={"policy": 1.0, "value": 1.0},
                metrics={"policy": "categorical_accuracy", "value": "mse"},
            )
            return model
        except Exception as e:
            print(e)
            return

    def plot_model(self):
        plot_model(self.model, show_shapes=True, show_layer_names=True)

    @abstractmethod
    def set_backbone(self):
        """
        Méthode abstraite pour définir le tronc du modèle (blocs convolutifs,
          etc.).

        Args: None

        Returns:
            None: Doit être implémentée dans une sous-classe.
        """
        raise NotImplementedError(
            "set_backbone() must be implemented in subclasses"
        )

    def set_model(self, verbose=False):
        self.model = self.create_policy_value_heads(verbose)

    # -------------------------------------
    # Entrainement et évaluation
    # -------------------------------------

    def log_accuracy(self, results_dict={}):
        """
        Enregistre la dernière précision de la tête "policy" dans un
          dictionnaire.

        Args:
            results_dict (dict): Dictionnaire auquel ajouter les résultats.

        Returns:
            None: Met à jour results_dict avec l'accuracy du modèle.
        """
        results_dict[self.__class__.__name__] = {
            "instance": self,
            "accuracy": self.policy_acc[-1],
        }

    def plot_training_history(self):
        """
        Affiche les courbes d'apprentissage pour les pertes et métriques
        (par époque).

        Args: None

        Returns:
            None: Affiche un graphique matplotlib.
        """
        epochs = range(1, len(self.loss_total) + 1)  # Liste des époques

        plt.figure(figsize=(12, 6))

        plt.suptitle(
            f"Évolution de l'entraînement du modèle : {self.name}", fontsize=16
        )

        # Graphique des pertes
        plt.subplot(1, 2, 1)
        plt.plot(epochs, self.loss_total, label="Loss totale", marker="o")
        plt.plot(epochs, self.policy_loss, label="Policy Loss", marker="o")
        plt.plot(epochs, self.value_loss, label="Value Loss", marker="o")
        plt.xlabel("Epochs")
        plt.ylabel("Valeur de la loss")
        plt.title("Évolution des losses")
        plt.legend()
        plt.grid(True)

        # Graphique des métriques
        plt.subplot(1, 2, 2)
        plt.plot(epochs, self.policy_acc, label="Policy Accuracy", marker="o")
        plt.plot(epochs, self.value_mse, label="Value MSE", marker="o")
        plt.xlabel("Epochs")
        plt.ylabel("Valeur des métriques")
        plt.title("Évolution des métriques")
        plt.legend()
        plt.grid(True)

        # Affichage
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()

    def train_model(self, batch=128, epochs=20, verbose=False):
        """
        Entraîne le modèle en plusieurs époques avec suivi des métriques.

        Args:
            batch (int): Taille du batch d'entraînement.
            epochs (int): Nombre total d'époques.

        Returns:
            None: Met à jour l'historique d'entraînement.
        """
        for i in range(1, epochs + 1):
            print(f"epoch {i}")
            golois.getBatch(
                self.input_data,
                self.policy,
                self.value,
                self.end,
                self.groups,
                i * self.N,
            )
            history = self.model.fit(
                self.input_data,
                {"policy": self.policy, "value": self.value},
                epochs=1,
                batch_size=batch,
            )
            # Extraction des valeurs depuis history.history
            self.loss_total.append(history.history["loss"][0])
            self.policy_loss.append(
                history.history["policy_loss"][0]
            )  # Policy loss
            self.value_loss.append(
                history.history["value_loss"][0]
            )  # Value loss
            self.policy_acc.append(
                history.history["policy_categorical_accuracy"][0]
            )  # Policy accuracy
            self.value_mse.append(history.history["value_mse"][0])
            if i % 5 == 0:
                gc.collect()
            if i % 20 == 0:
                golois.getValidation(
                    self.input_data, self.policy, self.value, self.end
                )
                val = self.model.evaluate(
                    self.input_data,
                    [self.policy, self.value],
                    verbose=0,
                    batch_size=batch,
                )
                if verbose:
                    print(f"{val=}")

    def workflow(
        self,
        epochs=20,
        batch=128,
        save_model=False,
        log_accuracy_dict={},
        verbose=False,
    ):
        """
        Exécute le flux de travail complet : construction, entraînement et évaluation du modèle.

        Args:
            epochs (int): Nombre d'époques pour l'entraînement.
            batch (int): Taille du batch pour l'entraînement.
            save_model (boolean): Si True enregistre le modèle au format h5 dans un fichier
            Nom du fichier pour enregistrer le modèle.
            log_accuracy_dict (dict): Dictionnaire pour enregistrer les résultats d'accuracy.

        Returns:
            None
        """
        self.set_backbone()
        self.set_model(verbose=verbose)
        self.train_model(epochs=epochs, batch=batch, verbose=verbose)
        if save_model:
            if not self.name:
                print(
                    "Le modèle n’a pas de nom défini, il ne pourra donc pas être "
                    "enregistré."
                )
            else:
                self.save_model(name=f"{self.name}.h5")
        self.plot_training_history()
        self.log_accuracy(results_dict=log_accuracy_dict)

    # -------------------------------------
    # Sauvegarde du modèle
    # -------------------------------------

    def save_model(self, name):
        """
        Sauvegarde le modèle et déclenche le téléchargement dans le fichier dédié.

        Args:
            name (str): Nom de fichier pour enregistrer le modèle (.h5, etc.)

        Returns:
            None
        """
        self.model.save(name)
        # A décommenter avec Google Colab
        files.download(name)

Dans l'objectif de conserver le modèle ayant obtenu la meilleur précision, une méthode `save_best_model` est également implémentée.
Celle-ci prend deux paramètres d'entrée :
- un dictionnaire intitulé `results_dict` qui contient les instances des modèles et leur précision respective ;
- le nom du fichier dans lequel sera conservé le modèle.

In [ ]:
def save_best_model(results_dict, model_name="test.h5"):
    """
    Sauvegarde le modèle ayant obtenu la meilleure précision (accuracy) à
    partir d'un dictionnaire de résultats.

    Le dictionnaire 'results_dict' doit contenir, pour chaque clef
    (nom du modèle), une structure :
        {
            "instance": instance_du_modèle,
            "accuracy": précision_obtenue (float)
        }

    La fonction identifie l'entrée avec la précision maximale, affiche un
    résumé, et sauvegarde l'instance correspondante au format Keras (.h5) sous
    le nom spécifié.

    Paramètres
    ----------
    results_dict : dict
        Dictionnaire contenant les modèles et leurs précisions associées.

    model_name : str
        Nom du fichier dans lequel sauvegarder le meilleur modèle (format .h5).

    Retourne
    --------
    None
    """
    if not results_dict:
        return None  # Gestion du cas où le dictionnaire est vide

    # Recherche le modèle avec la meilleure précision
    best_model_key = max(
        results_dict, key=lambda k: results_dict[k]["accuracy"]
    )

    # garder en mémoire la meilleure précision
    best_accuracy = results_dict[best_model_key]["accuracy"]

    # et l'instance du modèle
    best_instance = results_dict[best_model_key]["instance"]

    print(
        f"Le réseau {best_instance.name} est celui qui a enregistré la "
        f"meilleure accuracy : {best_accuracy}"
    )

    best_instance.save_model(model_name)

Un dictionnaire nommé `log_accuracy_dict` est initialisé pour enregistrer les *accuracies* de chaque réseau entraîné, constituant ainsi un historique des performances des différentes architectures.

In [ ]:
log_accuracy_dict = {}

Avant de nous pencher sur l’étude comparative des architectures telles que *ResNet*, *MobileNet*, *ConvNeXt* et *ShuffleNet*, il est essentiel de commencer par analyser le réseau de base fourni dans l’énoncé. Cette première étape permettra de mieux comprendre la structure attendue, de poser les fondations de notre pipeline d’entraînement et d’évaluation, et de justifier les choix effectués dans la conception de nos variantes.

Cette section servira également de point d’ancrage pour comparer les performances des différentes architectures que nous aborderons par la suite.

# Réseau de référence fourni dans l’énoncé

Dans cette section, nous analysons l’architecture minimale proposée dans l'énoncé, implémentée dans la classe `GONetDemo`. Ce modèle sert de référence de base pour les futures comparaisons. Il repose sur une pile de couches convolutionnelles simples avec activation ReLU.
Le but étant d'offrir un socle d'expérimentation rapide, sans complexité architecturale majeure, afin de valider le pipeline RL et tester la structure `GoNet`.

## Description de l’architecture

```python
class GONetDemo(GONet):

    def __init__(self):
        ...

    def set_backbone(self):
        """
        Définit l'architecture du tronc du réseau de neurones.

        Args: None

        Returns:
            None: Affecte self.x (sortie du tronc)
        """
        filters = 32
        x = layers.Conv2D(filters, 1, activation="relu", padding="same")(
            self.input_layer
        )
        for _ in range(5):
            x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
        self.x = x
```

Le tableau ci-dessous regroupe les Composants clefs du réseau.

| Étape               | Description                                                                          |
| ------------------- | ------------------------------------------------------------------------------------ |
| `Conv2D(1x1)`       | Réduction ou transformation linéaire initiale des canaux.                            |
| 5x `Conv2D(3x3)`    | Empilement de convolutions avec *padding* "same" pour préserver la dimension spatiale. |
| `activation="relu"` | Activation non-linéaire standard.                                                    |
| `filters=32`        | Taille constante des cartes de caractéristiques.                                     |

Il s’agit d’un réseau entièrement convolutionnel, sans *downsampling* explicite (pas de *pooling* ni *stride*), adapté à des images de petite taille ou des tâches où la précision spatiale est cruciale.

> Dans les réseaux de neurones convolutionnels, plusieurs mécanismes permettent de réduire la taille spatiale (hauteur $\times$ largeur) des cartes de caractéristiques : c'est ce qu'on appelle le **downsampling**. Cette opération est essentielle pour diminuer la complexité computationnelle, élargir le champ réceptif des neurones, et capturer des informations à plus grande échelle. Deux méthodes courantes de downsampling sont le **pooling** et l'utilisation de **convolutions avec stride**.
> Le **pooling** consiste à résumer localement une région d'activation, par exemple en prenant la valeur maximale (**MaxPooling**) ou la moyenne (**AveragePooling**) sur une fenêtre glissante. Quant au **stride**, il représente le pas de déplacement de cette fenêtre : un `stride=1` signifie un déplacement pixel par pixel, tandis qu’un `stride=2` fait sauter un pixel à chaque étape, divisant ainsi la résolution par deux. Ces techniques jouent un rôle central dans la conception d'architectures efficaces, notamment pour extraire des représentations hiérarchiques à partir d’images.

La figure ci-dessous illustre la structure du réseau de neurones associée à `GONetDemo`.

<p align="center">
  <img src="https://raw.githubusercontent.com/auduvignac/deep_learning_go/refs/heads/main/figures/gonetdemo.png?token=GHSAT0AAAAAADC7QGOOSBXO3RINXSBOJWHC2ATNXDQ" alt="GONet Demo" width="600"/>
</p>


> La construction d’un réseau de neurones convolutionnel avec `Keras` s’appuie sur la couche `Conv2D`, qui applique des filtres sur des données 2D, comme un plateau de Go. Après les couches convolutionnelles, la couche `Flatten` convertit la grille de sorties en un vecteur unidimensionnel, prêt à être exploité par une ou plusieurs couches `Dense`, typiquement utilisées pour la classification (ex. : prédiction d’un coup) ou la régression (ex. : évaluation d’une position).


## Intégration dans `GoNet`

`GONetDemo` hérite logiquement de `GONet`, qui définit le squelette général du modèle.

L’intégration consiste simplement à surcharger `set_backbone`, laissant `GONet` gérer l’assemblage complet du modèle.

## Remarques relatives à la structure du réseau `GONet`

* Ce réseau ne contient pas de mécanismes avancés (pas de normalisation, résidus, ni mobilisation de paramètres) ;
* Il est utile pour tester la cohérence fonctionnelle du cadre `GONet` (entrée/sortie, apprentissage) ;
* Il servira de *baseline* comparative vis-à-vis des architectures modernes (*ResNet*, *MobileNet*, *ConvNeXt* et *ShuffleNet*).


In [ ]:
class GONetDemo(GONet):

    def __init__(
        self,
        moves=361,
        N=10000,
        planes=31,
        max_params=100000,
        name="model",
        filters=32,
    ):
        super().__init__(
            moves=moves, N=N, planes=planes, max_params=max_params, name=name
        )
        self.filters = filters

    def set_backbone(self):
        """
        Définit l'architecture du tronc du réseau de neurones.

        Args: None

        Returns:
            None: Affecte self.x (sortie du tronc)
        """
        x = layers.Conv2D(self.filters, 1, activation="relu", padding="same")(
            self.input_layer
        )
        for _ in range(5):
            x = layers.Conv2D(
                self.filters, 3, activation="relu", padding="same"
            )(x)
        self.x = x

## Entraînement du modèle `GONetDemo`

Pour rappel, l'appel au *pipeline* d'entraînement se fait via la méthode `workflow`, qui encapsule les différentes étapes : préparation des données, entraînement, évaluation, et sauvegarde du modèle.

Les paramètres et leur description associée sont décrites dans le tableau ci-dessous.

| Paramètre           | Description                                             |
| ------------------- | ------------------------------------------------------- |
| `epochs`            | Nombre d'époques d'entraînement                         |
| `batch`             | Taille de lot utilisée pour le gradient                 |
| `name`              | Nom du fichier `.h5` pour la sauvegarde du modèle       |
| `log_accuracy_dict` | Dictionnaire pour enregistrer les précisions par modèle |

Le fichier qui contiendra le modèle associé à `GONetDemo` est intitulé : `gonetdemo.h5`.


In [ ]:
GONetDemo_instance = GONetDemo(name="GONetDemo")

GONetDemo_instance.workflow(batch=128, log_accuracy_dict=log_accuracy_dict)

## Interprétation des courbes d'entraînement

Analysons de manière détaillée les métriques et les courbres d'entraînement du modèle `GONetDemo`.

### Évaluation des pertes

La figure de gauche représente les courbes des différentes pertes :
- *Loss* totale : mesure globale de l'erreur sur les deux sorties (*policy* + *value*).
- *Policy loss* : erreur spécifique à la tête de *policy* (prédiction des coups).
- *Value loss* : erreur sur la prédiction de l'issue des parties (value head).

Les observations qui s'en dégagent sont les suivantes :

- Les **pertes sont quasiment constantes**, avec très peu d'évolution.
- Cela suggère que le **modèle n'apprend pas efficacement** :
  - soit les **gradients sont faibles ou bloqués** (vanishing gradients),
  - soit les **données ne permettent pas de progresser** (mauvais format, peu variées),
  - soit l'architecture est trop **limitée en capacité expressive**.

### Évaluation des métriques

Sur la figure de droite :
- *Policy accuracy* : mesure la capacité à prédire les bons coups ;
- *Value MSE* : mesure l'erreur quadratique sur la prédiction de l'issue.

Les observations qui s'en dégagent sont les suivantes :

- La **Policy accuracy** reste très faible (~0.005 à 0.007), sans progression.
- La **Value MSE** est stable autour de 0.12, ce qui confirme le **manque d'évolution du modèle**.

### Tableau récapitulatif des métriques

| Élément analysé     | Observation principale                                   |
|---------------------|----------------------------------------------------------|
| Loss totale         | Constante, absence d'optimisation                        |
| Policy Loss         | Stable, aucun apprentissage apparent                     |
| Value Loss          | Inchangée, ~0.7                                          |
| Policy Accuracy     | Très faible, aucune amélioration                         |
| Value MSE           | Stable (~0.12), peu d'évolution                          |

### Conclusion quant au modèle `GONetDemo`

Ce modèle a été conçu dans un objectif de validation d'implémentation. Bien que les performances observées restent modestes, les résultats obtenus permettent de valider le bon comportement de l'architecture et du *pipeline* d'entraînement.
Fort de cette conclusion, l'exploration d'architectures plus puissantes peut désormais être envisagée.
L'architecture de type *ResNet* sera étudiée en premier lieu.

# ResNet

Introduite pour remédier au problème d'évanescence du gradient dans les réseaux profonds, l'architecture des réseaux de neuronnes résiduels repose sur l'ajout de connexions résiduelles, permettant l'entraînement efficace de modèles plus profonds et expressifs — un atout particulièrement pertinent dans le contexte du jeu de Go.

## Quelques rappels

« Un réseau neuronal résiduel ou réseau de neurones résiduel est une architecture de réseaux de neurones caractérisée par l'emploi de connexions résiduelles ou connexions saute-couches.

ResNet est un nom propre qui désigne le réseau neuronal résiduel qui a remporté la compétition ILSVRC en 2015.

Un réseau de neurones résiduel (ResNet) est un réseau de neurones artificiels (artificial neuronal network) qui s'appuie sur des constructions connues à partir de cellules pyramidales du cortex cérébral. Pour ce faire, les réseaux de neurones résiduels utilisent des connexions saute-couches, i.e. des raccourcis, pour sauter par-dessus certaines couches. » (cf. https://datafranca.org/wiki/R%C3%A9seau_neuronal_r%C3%A9siduel).

Le principe des réseaux résiduels consiste à ajouter l'entrée d'une couche à sa sortie. Grâce à cette simple modification, l'entraînement est plus rapide et permet la construction de réseaux plus profonds.
L'utilisation des réseaux résiduels pour le Go permet d'accélérer l'apprentissage, d'augmenter la précision et de permettre l'entraînement de réseaux plus profonds (cf. https://www.lamsade.dauphine.fr/~cazenave/papers/resnet.pdf).

L'implémentation du réseau de neurones résiduel implémenté s'appuie sur celle réalisée dans https://www.lamsade.dauphine.fr/~cazenave/papers/resnet.pdf.


## Description de l'architecture

L'architecture `GONetResNet` repose sur le principe fondamental des connexions résiduelles, qui permettent de faciliter l'entraînement de réseaux plus profonds.

L'implémentation d'un bloc résiduel est donné ci-dessous.

```python
def residual_block(self, x):
    filters = 32
    conv_5x5 = layers.Conv2D(filters, 5, padding="same")(x)
    conv_1x1 = layers.Conv2D(filters, 1, padding="same")(x)
    added = layers.Add()([conv_5x5, conv_1x1])
    return layers.Activation("relu")(added)
```

Chaque bloc résiduel est constitué de deux chemins parallèles :
- Une convolution 5 $\times$ 5 pour extraire des caractéristiques complexes ;
- Une convolution 1 $\times$ 1 jouant le rôle de raccourci (skip connection) ;
- Les deux sorties sont additionnées puis passées à une activation ReLU.

La méthode `set_backbone` empile `num_blocks` blocs résiduels. Par défaut, `num_blocks` est instanciée à 1 dans le constructeur.

```python
def set_backbone(self):
    x = self.input_layer
    for _ in range(self.num_blocks):
        x = self.residual_block(x)
    self.x = x
```

> Cette implémentation reprend l'esprit de *ResNet*, bien qu'elle ne reproduise pas rigoureusement les blocs standards (pas de *BatchNormalization*, pas de projection si changement de dimension).

In [ ]:
class GONetResNet(GONet):

    def __init__(
        self,
        moves=361,
        N=10000,
        planes=31,
        max_params=100000,
        name="model",
        num_blocks=1,
        filters=32,
    ):
        super().__init__(
            moves=moves, N=N, planes=planes, max_params=max_params, name=name
        )
        self.num_blocks = num_blocks
        self.filters = filters

    def residual_block(self, x):
        shortcut = x
        x = layers.Conv2D(self.filters, 3, padding="same")(x)
        x = layers.ReLU()(x)
        x = layers.Conv2D(self.filters, 3, padding="same")(x)

        # Projection du shortcut si nécessaire
        if shortcut.shape[-1] != x.shape[-1]:
            shortcut = layers.Conv2D(self.filters, 1, padding="same")(shortcut)

        x = layers.Add()([x, shortcut])
        return layers.ReLU()(x)

    def set_backbone(self):
        x = self.input_layer
        for _ in range(self.num_blocks):
            x = self.residual_block(x)
        self.x = x

## Entraînement du modèle `GONetResNet`

Dans une démarche analogue à celle appliquée sur le réseau `GONetDemo`, l'entraînement est lancé sur 10 *epochs* avec enregistrement du modèle et suivi dans le dictionnaire `log_accuracy_dict`.

In [ ]:
GONetResNet_instance = GONetResNet(name="GONetResNet_1_layer")

GONetResNet_instance.workflow(
    batch=128,
    log_accuracy_dict=log_accuracy_dict,
)

## Interprétation des courbes d'entraînement

Analysons de manière détaillée les métriques et les courbes d'entraînement du modèle `GONetResNet`.

### Évolution des *losses*

Le tableau ci-dessous présente les métriques associées aux *Losses* et leur interprétation respective.

| Courbe          | Observation                                                                       |
| --------------- | --------------------------------------------------------------------------------- |
| *Loss* totale | Baisse nette constatée, indiquant une phase de convergence.           |
| *Policy Loss* | En diminution progressive : le modèle apprend à mieux prédire les coups.          |
| *Value Loss*  | Stable mais légèrement inférieure à celle de `GONetDemo` : valeur mieux capturée. |

L'ajout d'une connexion résiduelle semble améliorer la stabilité de l'apprentissage et facilite la descente de la *loss policy*, contrairement à `GONetDemo` où les *losses* restaient quasi constantes.

### Évolution des métriques

Le tableau ci-dessous présente les métriques permettant de quantifier des prédictions obtenues après l'entraînement.

| Courbe              | Observation                                                                                                                                                       |
| ------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| *Policy Accuracy* | Amélioration progressive. Au bout de 20 *epoch* atteint 14%, résultat en nette progression par rapport à `GONetDemo` |
| **Value MSE**       | Stable autour de 0.12 : peu de gain ici, mais pas de régression non plus.                                                                                         |

Les éléments qui se dégagent de ces résultats sont les suivants :

- La *Policy Accuracy* montre que le modèle est en mesure d'extraire des *patterns* pertinents ;
- La *Value MSE* reste globalement inchangée. Ce qui suggère possiblement un manque de capacité ou du bruit dans les données de valeur.

### Conclusion quant aux résultats obtenus

- L'architecture `GONetResNet`, bien que simple dans sa construction (une seulles couche résiduelle), permet une *meilleure convergence* que `GONetDemo`.
- La baisse des *losses*, notamment sur la politique, ainsi que la progression de l'*accuracy* montrent que le modèle apprend effectivement à identifier des coups probables ;
- Ces résultats soulignent l'intérêt des connexions résiduelles dans l'entraînement de réseaux plus profonds, en particulier dans le contexte du Go où les relations spatiales complexes dominent.

Ces résultats encouragent à explorer un nombre croissant de couches résiduelles (paramétrables via `num_blocks`). Il faut toutefois prendre garde à ne pas dépasser le nombre de paramètres autorisés, 100 000 en l'occurrence.

## Analyse quantitative de l'augmentation du nombre de blocs résiduels sur l'efficacité du réseau

La première étape consiste à identifier le nombre maximal de couches résiduelles permettant de rester en dessous du seuil de 100 000 paramètres.
Une fois cette contrainte établie, les modèles correspondants seront entraînés, afin d'analyser dans quelle mesure l'augmentation du nombre de couches résiduelles influence les performances.


In [ ]:
num_blocks = 1

while True:
    GONetResNet_instance = GONetResNet(num_blocks=num_blocks)
    GONetResNet_instance.set_backbone()
    GONetResNet_instance.set_model()

    if GONetResNet_instance.nb_params > GONetResNet_instance.max_params:
        print(
            f"Nombre de paramètres dépassé ({GONetResNet_instance.nb_params}).\n"
            "Nombre maximal de blocs résiduels sans dépasser 100 000 : "
            f"{num_blocks - 1}"
        )
        break
    num_blocks += 1

Afin de respecter la contrainte de complexité ($\leq$ 100 000 paramètres), le nombre de blocs résiduels a été limité à 3, valeur maximale avant dépassement.

In [ ]:
# Liste pour stocker les résultats
GONetResNet_instances = []

# Fonction de création + entraînement d'un modèle GONetResNet


def run_gonetresnet(num_blocks):
    model_name = f"gonetresnet_{num_blocks}_blocks"
    print(f"Démarrage de l'entraînement : {model_name}")

    # Création de l'instance avec un nombre variable de blocs
    instance = GONetResNet(num_blocks=num_blocks, name=model_name)
    instance.set_backbone()
    instance.set_model()
    instance.train_model(epochs=20, batch=128)  # adapte epochs si besoin

    print(f"Modèle terminé : {model_name}")
    return {"num_blocks": num_blocks, "instance": instance}


# Entraînement séquentiel avec barre de progression
for i in tqdm(range(1, 4), desc="Entraînement des modèles"):
    result = run_gonetresnet(i)
    GONetResNet_instances.append(result)

In [ ]:
for GONetResNet_instance in GONetResNet_instances:
    instance = GONetResNet_instance.get("instance")
    instance.plot_training_history()
    instance.log_accuracy(results_dict=log_accuracy_dict)

epochs = range(1, 21)

plt.figure(figsize=(10, 6))
for i, inst_dict in enumerate(GONetResNet_instances):
    instance = inst_dict.get("instance")
    if instance is not None and hasattr(instance, "policy_acc"):
        plt.plot(
            epochs, instance.policy_acc, marker="o", label=f"{instance.name}"
        )

plt.xlabel("Epochs")
plt.ylabel("Policy Accuracy")
plt.title("Comparaison de l'accuracy")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## Analyse comparative de l'effet du nombre de blocs résiduels

### *Policy Accuracy*

| Architecture        | *Accuracy* finale | Convergence |
|---------------------|-------------------|-------------|
| 1 bloc résiduel     | ~0.133            | Lente       |
| 2 blocs résiduels   | ~0.206            | Rapide      |
| 3 blocs résiduels   | ~0.210            | Très rapide |

- Le passage de 1 à 2 blocs apporte un gain significatif en *accuracy* et en vitesse d'apprentissage ;
- Le passage de 2 à 3 blocs ne procure qu'un gain marginal ;
- À partir de 2 blocs, l'amélioration de la policy accuracy sature.

### *Losses* et *Value MSE*

- *Policy loss* diminue régulièrement pour les 3 modèles, plus efficacement à partir de 2 blocs ;
- *Value loss* et *Value MSE* restent stables et peu sensibles à l'augmentation de la profondeur, autour de ~0.12-0.13.
- Cela suggère une difficulté à optimiser la valeur indépendamment de la complexité du réseau.

### Conclusion quant à l'analyse quantitative du nombre croissant de couches résiduelles

Le modèle à deux blocs résiduels apparaît comme le meilleur compromis :
- Excellente *accuracy* atteinte rapidement ;
- Complexité contrôlée (moins de surcoût que 3 blocs) ;
- Pas de gain notable à utiliser un troisème bloc.

Le modèle à deux blocs résiduels constitue donc un compromis optimal, alliant expressivité et stabilité, sans dépasser les contraintes fixées.

Cette étude a mis en exergue que l'ajout de blocs résiduels améliore significativement la performance des réseaux de neurones convolutifs, jusqu'à un certain point. Toutefois, cette amélioration s'accompagne d'une augmentation du nombre de paramètres, du temps d'entraînement et de la consommation mémoire.

Dans une optique d'optimisation du rapport performance et d'efficacité, il devient pertinent de se tourner vers des architectures plus légères et efficaces.

C'est dans ce contexte que s'inscrit *MobileNet*, une architecture conçue pour offrir une bonne précision tout en réduisant fortement la complexité computationnelle, grâce à l'utilisation de **convolutions séparables en profondeur**.

La prochaine étape consiste à étudier cette architecture et la comparer avec les architectures étudiées jusqu'à présent.

# MobileNet

Les réseaux *MobileNet* sont couramment utilisés en vision par ordinateur pour classer des images. Ils atteignent une haute précision sur les jeux de données standards tout en conservant un nombre de paramètres inférieur à celui d'autres architectures de réseaux neuronaux.

Etudions cette architecture sur le jeu de données fourni.


In [ ]:
class GONetMobileNet(GONet):

    def __init__(
        self,
        moves=361,
        N=10000,
        planes=31,
        max_params=100000,
        name="model",
        num_blocks=1,
        trunk=32,
        filters=32,
    ):
        super().__init__(
            moves=moves, N=N, planes=planes, max_params=max_params, name=name
        )
        self.num_blocks = num_blocks
        self.trunk = trunk
        self.filters = filters

    def bottleneck_block(self, x):
        m = layers.Conv2D(
            self.filters,
            (1, 1),
            kernel_regularizer=regularizers.l2(0.0001),
            use_bias=False,
        )(x)
        m = layers.BatchNormalization()(m)
        m = layers.Activation("relu")(m)
        m = layers.DepthwiseConv2D(
            (3, 3),
            padding="same",
            kernel_regularizer=regularizers.l2(0.0001),
            use_bias=False,
        )(m)
        m = layers.BatchNormalization()(m)
        m = layers.Activation("relu")(m)
        m = layers.Conv2D(
            self.trunk,
            (1, 1),
            kernel_regularizer=regularizers.l2(0.0001),
            use_bias=False,
        )(m)
        m = layers.BatchNormalization()(m)
        return layers.Add()([m, x])

    def set_backbone(self):
        x = self.input_layer
        x = layers.Conv2D(
            self.trunk,
            1,
            padding="same",
            kernel_regularizer=regularizers.l2(0.0001),
        )(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        for _ in range(self.num_blocks):
            x = self.bottleneck_block(x)
        self.x = x

In [ ]:
GONetMobileNet_instance = GONetMobileNet(name="GONetMobileNet")
GONetMobileNet_instance.set_backbone()
GONetMobileNet_instance.set_model()
GONetMobileNet_instance.save_model("GONetMobileNet.h5")

In [ ]:
GONetMobileNet_instance = GONetMobileNet(name="GONetMobileNet")
GONetMobileNet_instance.workflow(
    batch=128,
    log_accuracy_dict=log_accuracy_dict,
)

In [ ]:
num_blocks = 1

while True:
    GONetMobileNet_instance = GONetMobileNet(num_blocks=num_blocks)
    GONetMobileNet_instance.set_backbone()
    GONetMobileNet_instance.set_model()

    if GONetMobileNet_instance.nb_params > GONetMobileNet_instance.max_params:
        print(
            f"Nombre de paramètres dépassé ({GONetMobileNet_instance.nb_params}).\n"
            "Nombre maximal de blocs résiduels sans dépasser 100 000 : "
            f"{num_blocks - 1}"
        )
        break
    num_blocks += 1

In [ ]:
# Liste pour stocker les résultats
GONetMobileNet_instances = []

# Fonction de création + entraînement d'un modèle GONetResNet


def run_gomobilenet(num_blocks):
    model_name = f"gomobilenet_{num_blocks}_blocks"
    print(f"Démarrage de l'entraînement : {model_name}")

    # Création de l'instance avec un nombre variable de blocs
    instance = GONetMobileNet(num_blocks=num_blocks, name=model_name)
    instance.set_backbone()
    instance.set_model()
    instance.train_model(epochs=20, batch=128)

    print(f"Modèle terminé : {model_name}")
    return {"num_blocks": num_blocks, "instance": instance}


# Entraînement séquentiel avec barre de progression
for i in tqdm(range(1, 29), desc="Entraînement des modèles"):
    result = run_gomobilenet(i)
    GONetMobileNet_instances.append(result)

In [ ]:
def plot_metric(instances, attribute, ylabel, title, epochs=range(1, 21)):
    plt.figure(figsize=(12, 6))

    for inst_dict in instances:
        instance = inst_dict.get("instance")
        if instance is not None and hasattr(instance, attribute):
            plt.plot(
                epochs,
                getattr(instance, attribute),
                marker="o",
                label=f"{instance.name}",
            )

    plt.xlabel("Epochs")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True)

    plt.legend(
        bbox_to_anchor=(1.05, 1),
        loc="upper left",
        borderaxespad=0.0,
        fontsize="small",
        title="Modèles",
    )

    plt.tight_layout(rect=[0, 0, 0.75, 1])
    plt.show()

In [ ]:
plot_metric(
    GONetMobileNet_instances,
    "policy_acc",
    "Policy Accuracy",
    "Comparaison de l'accuracy",
)
plot_metric(
    GONetMobileNet_instances,
    "loss_total",
    "Loss totale",
    "Comparaison de la loss totale",
)
plot_metric(
    GONetMobileNet_instances,
    "policy_loss",
    "Policy Loss",
    "Comparaison de la policy loss",
)
plot_metric(
    GONetMobileNet_instances,
    "value_loss",
    "Value Loss",
    "Comparaison de la value loss",
)
plot_metric(
    GONetMobileNet_instances, "value_mse", "Value MSE", "Comparaison du MSE"
)

In [ ]:
def extract_final_metrics(instances, metric_names):
    metrics_data = []

    for inst_dict in instances:
        instance = inst_dict.get("instance")
        if instance is not None:
            row = {
                "name": instance.name,
                "num_blocks": inst_dict.get("num_blocks"),
            }
            for metric in metric_names:
                values = getattr(instance, metric, None)
                if values:
                    row[f"final_{metric}"] = values[-1]
            metrics_data.append(row)

    df = pd.DataFrame(metrics_data)
    return df

In [ ]:
# Liste des métriques à extraire
metric_list = [
    "loss_total",
    "policy_loss",
    "value_loss",
    "policy_acc",
    "value_mse",
]

# Création du DataFrame
df_metrics = extract_final_metrics(GONetMobileNet_instances, metric_list)

# Tri par la métrique de précision
df_metrics = df_metrics.sort_values(
    by="final_policy_acc", ascending=False
).reset_index(drop=True)

display(df_metrics.style.format(precision=4))

## Interprétation des résultats obtenus

L'analyse porte sur les performances de 28 modèles `GONetMobileNet` variant par le nombre de blocs (de 1 à 28). Les métriques principales évaluées sont les pertes (*losses*) et les performances (métriques) des têtes `policy` et `value`. Les observations suivantes synthétisent le comportement global observé durant l'entraînement.

### Évolution des losses

- *Policy Loss* : elle décroît de manière régulière au fil des époques pour toutes les architectures, indiquant un apprentissage progressif de la politique.
- **Value Loss** : elle diminue initialement, puis stagne autour de **0.70** à partir de **5 epochs**, quel que soit le nombre de blocs ;
- *Loss totale* : la courbe suit globalement la même tendance que la somme des deux pertes précédentes, avec une décroissance initiale avant stabilisation.

### Évolution des métriques

- *Policy Accuracy* : la précision augmente légèrement au début mais reste modeste dans l'ensemble. Elle atteint son maximum (0.2455) avec **5 blocs**, puis décline lentement avec l'augmentation de la profondeur ;
- *Value MSE* : une diminution progressive est observée sur les premières époques, pour ensuite stagner autour de **0.12**, sans amélioration significative liée à la profondeur.

### Conclusion

- Les **meilleures performances globales** sont obtenues avec des architectures comprenant **entre 4 et 6 blocs**. En s'appuyant exclusivement sur l'*accurracy* le meilleur se trouve être celui comprenant **5 blocs** ;
- L'**augmentation du nombre de blocs** au-delà de ce seuil n'apporte **aucun bénéfice clair**, ni sur les pertes ni sur les métriques, et tend même à les dégrader ;
- Le **MSE et la value loss stagnants** indiquent que la tête `value` reste limitée, probablement du fait de la nature simulée des données ;

Après avoir exploré l'architecture MobileNet avec différentes profondeurs, il est pertinent d'élargir la comparaison à des modèles plus récents et puissants.

Parmi ceux-ci, *ConvNeXt* s'impose comme une évolution majeure des réseaux convolutifs classiques. Inspiré des meilleures pratiques des Transformers, *ConvNeXt* revisite la conception des réseaux de neurones de convolution pour les rendre plus compétitifs sur les tâches de vision.

Nous allons désormais explorer cette architecture, en l'adaptant à notre classe `GoNet`, et en comparant ses performances aux réseaux précédemment étudiés.

# ConvNeXt

In [ ]:
class GONetConvNeXtLike(GONet):

    def __init__(
        self,
        moves=361,
        N=10000,
        planes=31,
        max_params=100000,
        name="model",
        num_blocks=1,
        trunk=96,
        filters=48,
    ):
        super().__init__(
            moves=moves, N=N, planes=planes, max_params=max_params, name=name
        )
        self.num_blocks = num_blocks  # nombre de blocs ConvNeXt
        self.trunk = trunk  # profondeur / projection initiale
        self.filters = filters  # nombre de filtres dans les blocs

    def set_backbone(self):
        """
        Définit un tronc ConvNeXt-like basé sur des blocs V1.

        Architecture :
        - Projection initiale vers 'filters' canaux
        - Répétition de 'num_blocks' blocs ConvNeXt
        - Réduction finale à 32 canaux pour compatibilité avec les têtes
        """
        x = self.input_layer

        # Projection initiale (31 → filters canaux)
        x = layers.Conv2D(self.filters, kernel_size=1, padding="same")(x)

        for _ in range(self.num_blocks):
            residual = x
            x1 = layers.DepthwiseConv2D(
                kernel_size=7, padding="same", use_bias=False
            )(x)
            x1 = layers.LayerNormalization(epsilon=1e-6)(x1)
            x1 = layers.Conv2D(
                4 * self.filters,
                kernel_size=1,
                activation="gelu",
                padding="same",
            )(x1)
            x1 = layers.Conv2D(self.filters, kernel_size=1, padding="same")(x1)
            x = layers.Add()([x, x1])
            x = layers.BatchNormalization()(x)

        # Réduction finale à 32 canaux pour create_policy_value_heads
        x = layers.Conv2D(32, kernel_size=1, padding="same", use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation("relu")(x)

        self.x = x

In [ ]:
GONetConvNeXtLike_instance = GONetConvNeXtLike(name="GONetConvNeXtLike")
GONetConvNeXtLike_instance.workflow(
    batch=128,
    log_accuracy_dict=log_accuracy_dict,
)

In [ ]:
num_blocks = 1

while True:
    GONetConvNeXtLike_instance = GONetConvNeXtLike(num_blocks=num_blocks)
    GONetConvNeXtLike_instance.set_backbone()
    GONetConvNeXtLike_instance.set_model()

    if (
        GONetConvNeXtLike_instance.nb_params
        > GONetConvNeXtLike_instance.max_params
    ):
        print(
            f"Nombre de paramètres dépassé ({GONetConvNeXtLike_instance.nb_params}).\n"
            "Nombre maximal de blocs résiduels sans dépasser 100 000 : "
            f"{num_blocks - 1}"
        )
        break
    num_blocks += 1

In [ ]:
# Liste pour stocker les résultats
GONetConvNeXtLike_instances = []

# Fonction de création + entraînement d'un modèle GONetResNet


def run_goconvnextlike(num_blocks):
    model_name = f"goconvnextlike_{num_blocks}_blocks"
    print(f"Démarrage de l'entraînement : {model_name}")

    # Création de l'instance avec un nombre variable de blocs
    instance = GONetConvNeXtLike(num_blocks=num_blocks, name=model_name)
    instance.set_backbone()
    instance.set_model()
    instance.train_model(epochs=20, batch=128)

    print(f"Modèle terminé : {model_name}")
    return {"num_blocks": num_blocks, "instance": instance}


# Entraînement séquentiel avec barre de progression
for i in tqdm(range(1, 4), desc="Entraînement des modèles"):
    result = run_goconvnextlike(i)
    GONetConvNeXtLike_instances.append(result)

In [ ]:
plot_metric(
    GONetConvNeXtLike_instances,
    "policy_acc",
    "Policy Accuracy",
    "Comparaison de l'accuracy",
)
plot_metric(
    GONetConvNeXtLike_instances,
    "loss_total",
    "Loss totale",
    "Comparaison de la loss totale",
)
plot_metric(
    GONetConvNeXtLike_instances,
    "policy_loss",
    "Policy Loss",
    "Comparaison de la policy loss",
)
plot_metric(
    GONetConvNeXtLike_instances,
    "value_loss",
    "Value Loss",
    "Comparaison de la value loss",
)
plot_metric(
    GONetConvNeXtLike_instances, "value_mse", "Value MSE", "Comparaison du MSE"
)

In [ ]:
# Liste des métriques à extraire
metric_list = [
    "loss_total",
    "policy_loss",
    "value_loss",
    "policy_acc",
    "value_mse",
]

# Création du DataFrame
df_metrics = extract_final_metrics(GONetConvNeXtLike_instances, metric_list)

# Tri par la métrique de précision
df_metrics = df_metrics.sort_values(
    by="final_policy_acc", ascending=False
).reset_index(drop=True)

display(df_metrics.style.format(precision=4))

# ShuffleNet

*ShuffleNet* est une architecture de réseau neuronal conçue pour être rapide et efficace. Elle repose sur le concept de permutation des canaux du tenseur d'entrée, ce qui améliore l'efficacité en termes de calcul et d'utilisation de la mémoire.

Le réseau se compose de deux principales parties :

1.  Couche de convolution : Cette couche a pour rôle d'extraire les caractéristiques du tenseur d'entrée.

2.  Couche de permutation (*shuffling*) : Cette couche permutent les canaux du tenseur d'entrée. Elle est conçue pour être légère et efficace, ce qui contribue fortement à la performance globale et à l'efficacité du réseau

*Shufflenet* est un mobilenet avec moins de paramètres puisqu'il y a des plans séparés.

In [ ]:
class GONetShuffleNet(GONet):
    def __init__(
        self,
        moves=361,
        N=10000,
        planes=31,
        max_params=100000,
        name="gonet_shufflenet",
        num_blocks=1,
        trunk=32,
        filters=32,
        shuffle_groups=4,
    ):
        super().__init__(moves, N, planes, max_params, name)
        self.num_blocks = num_blocks
        self.trunk = trunk
        self.filters = filters
        self.shuffle_groups = shuffle_groups

    def set_backbone(self):
        x = Conv2D(
            self.trunk,
            1,
            padding="same",
            kernel_regularizer=regularizers.l2(0.0001),
        )(self.input_layer)
        x = BatchNormalization()(x)
        x = ReLU()(x)

        for _ in range(self.num_blocks):
            x = self.bottleneck_block(
                x, expand=self.filters * 3, squeeze=self.trunk
            )

        self.x = x

    def bottleneck_block(self, x, expand, squeeze):
        residual = x
        x = self.gconv(x, channels=expand, shuffle_groups=self.shuffle_groups)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = self.channel_shuffle(x, self.shuffle_groups)
        x = DepthwiseConv2D(kernel_size=3, padding="same", use_bias=False)(x)
        x = BatchNormalization()(x)
        x = self.gconv(x, channels=squeeze, shuffle_groups=self.shuffle_groups)
        x = BatchNormalization()(x)
        x = Add()([residual, x])
        return ReLU()(x)

    def gconv(self, x, channels, shuffle_groups):
        input_ch = x.shape[-1]
        group_ch = input_ch // shuffle_groups
        out_ch = channels // shuffle_groups
        group_outputs = []
        for i in range(shuffle_groups):
            slice_x = x[:, :, :, i * group_ch : (i + 1) * group_ch]
            conv = Conv2D(out_ch, kernel_size=1, use_bias=False)(slice_x)
            group_outputs.append(conv)
        return Concatenate(axis=-1)(group_outputs)

    def channel_shuffle(self, x, shuffle_groups):
        """
        Applique l'opération de channel shuffle.
        Args:
            x: Tensor de forme [batch_size, height, width, channels]
            shuffle_groups: Nombre de groupes à créer
        Returns:
            Tensor après permutation des canaux entre les groupes
        """
        batch_size = tf.shape(x)[0]
        height = tf.shape(x)[1]
        width = tf.shape(x)[2]
        channels = tf.shape(x)[3]

        # Vérification statique de la divisibilité
        if x.shape[-1] is not None and x.shape[-1] % shuffle_groups != 0:
            raise ValueError(
                "Le nombre de canaux doit être divisible par le nombre de groupes pour le channel shuffle."
            )

        group_channels = channels // shuffle_groups

        # Reshape pour grouper les canaux
        x = tf.reshape(
            x, [batch_size, height, width, shuffle_groups, group_channels]
        )

        # Permutation des groupes et des canaux
        x = tf.transpose(
            x, [0, 1, 2, 4, 3]
        )  # (batch, height, width, group_channels, groups)

        # Flatten à nouveau en (batch, height, width, channels)
        x = tf.reshape(x, [batch_size, height, width, channels])

        return x

In [ ]:
GONetShuffleNet_instance = GONetShuffleNet(name="GONetShuffleNet")
GONetShuffleNet_instance.workflow(
    batch=128,
    log_accuracy_dict=log_accuracy_dict,
)

In [ ]:
num_blocks = 1

while True:
    GONetShuffleNet_instance = GONetShuffleNet(num_blocks=num_blocks)
    GONetShuffleNet_instance.set_backbone()
    GONetShuffleNet_instance.set_model()

    if (
        GONetShuffleNet_instance.nb_params
        > GONetShuffleNet_instance.max_params
    ):
        print(
            f"Nombre de paramètres dépassé ({GONetShuffleNet_instance.nb_params}).\n"
            "Nombre maximal de blocs résiduels sans dépasser 100 000 : "
            f"{num_blocks - 1}"
        )
        break
    num_blocks += 1

In [ ]:
# Liste pour stocker les résultats
GONetShuffleNet_instances = []


def run_goshufflenet(num_blocks):
    model_name = f"goshufflenet_{num_blocks}_blocks"
    print(f"Démarrage de l'entraînement : {model_name}")

    # Création de l'instance avec un nombre variable de blocs
    instance = GONetShuffleNet(num_blocks=num_blocks, name=model_name)
    instance.set_backbone()
    instance.set_model()
    instance.train_model(epochs=20, batch=128)

    print(f"Modèle terminé : {model_name}")
    return {"num_blocks": num_blocks, "instance": instance}


# Entraînement séquentiel avec barre de progression
for i in tqdm(range(1, 25), desc="Entraînement des modèles"):
    result = run_goshufflenet(i)
    GONetShuffleNet_instances.append(result)

In [ ]:
plot_metric(
    GONetShuffleNet_instances,
    "policy_acc",
    "Policy Accuracy",
    "Comparaison de l'accuracy",
)
plot_metric(
    GONetShuffleNet_instances,
    "loss_total",
    "Loss totale",
    "Comparaison de la loss totale",
)
plot_metric(
    GONetShuffleNet_instances,
    "policy_loss",
    "Policy Loss",
    "Comparaison de la policy loss",
)
plot_metric(
    GONetShuffleNet_instances,
    "value_loss",
    "Value Loss",
    "Comparaison de la value loss",
)
plot_metric(
    GONetShuffleNet_instances, "value_mse", "Value MSE", "Comparaison du MSE"
)

In [ ]:
# Liste des métriques à extraire
metric_list = [
    "loss_total",
    "policy_loss",
    "value_loss",
    "policy_acc",
    "value_mse",
]

# Création du DataFrame
df_metrics = extract_final_metrics(GONetShuffleNet_instances, metric_list)

# Tri par la métrique de précision
df_metrics = df_metrics.sort_values(
    by="final_policy_acc", ascending=False
).reset_index(drop=True)

display(df_metrics.style.format(precision=4))

# Convolutions en croix (*Cross-Shaped Convolutions*)

L'objectif de cette section est d'explorer une architecture basée sur les convolutions en croix. Pour cela, nous allons nous appuyer sur la classe : `GONetCrossLayer`.

In [ ]:
class GONetCrossLayer(GONet):
    def __init__(
        self,
        moves=361,
        N=10000,
        planes=31,
        max_params=100000,
        name="gonet_crosslayer",
        filters=16,
    ):
        super().__init__(moves, N, planes, max_params, name)
        self.filters = filters

    def set_backbone(self):
        # Branche gauche
        left = layers.Conv2D(
            32, 7, padding="same", kernel_regularizer=regularizers.l2(0.0001)
        )(self.input_layer)
        left = layers.BatchNormalization()(left)
        left = layers.ReLU()(left)

        left = layers.Conv2D(
            8, 1, padding="same", kernel_regularizer=regularizers.l2(0.0001)
        )(left)
        left = layers.BatchNormalization()(left)
        left = layers.ReLU()(left)

        left = self.cross_layer(left, width=1, filters=8)

        left = layers.Conv2D(
            32, 1, padding="same", kernel_regularizer=regularizers.l2(0.0001)
        )(left)

        # Branche droite
        right = layers.Conv2D(
            8, 1, padding="same", kernel_regularizer=regularizers.l2(0.0001)
        )(self.input_layer)
        right = layers.BatchNormalization()(right)
        right = layers.ReLU()(right)

        right = self.cross_layer(right, width=5, filters=8)

        right = layers.Conv2D(
            32, 1, padding="same", kernel_regularizer=regularizers.l2(0.0001)
        )(right)

        # Fusion
        x = layers.Add()([left, right])
        x = layers.ReLU()(x)

        self.x = x

    def cross_layer(self, x, width=1, filters=16):
        """
        Implémente une couche convolutive "en croix" via deux convolutions séparables :
        - verticale : (k, 1)
        - horizontale : (1, k)
        """
        vertical = layers.Conv2D(
            filters, (width, 1), padding="same", activation="relu"
        )(x)
        horizontal = layers.Conv2D(
            filters, (1, width), padding="same", activation="relu"
        )(x)
        return layers.Add()([vertical, horizontal])

In [ ]:
GONetCrossLayer_instance = GONetCrossLayer(name="GONetCrossLayer")
GONetCrossLayer_instance.set_backbone()
GONetCrossLayer_instance.set_model()
GONetCrossLayer_instance.save_model("GONetCrossLayer.h5")

In [ ]:
GONetCrossLayer_instance = GONetCrossLayer(name="GONetCrossLayer")
GONetCrossLayer_instance.workflow(
    batch=128,
    log_accuracy_dict=log_accuracy_dict,
)

Au cours des 20 époques d'entraînement, la loss totale du modèle `GONetCrossLayer` a diminué de manière continue, passant d'environ 6.5 à une valeur finale de 4.2796. Cette tendance indique une amélioration générale de l'optimisation du modèle. En parallèle, la loss value est restée stable autour de 0.69, avec une valeur finale de 0.6927, traduisant une difficulté persistante à améliorer cette composante durant l'apprentissage.

La policy loss, en revanche, a nettement baissé, passant de près de 5.9 à 3.5690, signalant une progression efficace sur cet aspect du modèle. Cette dynamique est cohérente avec l'évolution de la MSE, qui reste relativement constant et atteint 0.1210 en fin d'entraînement, confirmant l'absence d'amélioration notable sur la tête de valeur.

Enfin, la précision catégorielle de la politique a montré une nette progression, atteignant 0.2162 après un départ très faible. Cela reflète une capacité croissante du modèle à reproduire fidèlement les coups issus des données d'entraînement, et met en évidence la solidité de l'architecture GONetCrossLayer pour l'apprentissage de la politique.

Sur la base des différents résultats obtenus, abordons l'étude des meilleurs architectures en augmentant le nombre d'époques.

# Choix du modèle final

Le tableau suivant regroupe l'accuracy des meilleurs modèles retenues entraînés sur 50 époques.

| Architecture                    | Accuracy finale (%) |
|--------------------------------|---------------------|
| GONetShuffleNet_4_blocs        | 27.85               |
| GONetConvNeXtLike_2_blocs      | 27.65               |
| GONetMobileNet_5_blocs         | 27.51               |
| GONetResNet_3_blocs            | 26.83               |
| GONetCrossLayer                | 26.17               |

In [ ]:
# Définition des meilleures architectures à entraîner
instances = [
    GONetResNet(num_blocks=3, name="GONetResNet_3_blocs"),
    GONetMobileNet(num_blocks=5, name="GONetMobileNet_5_blocs"),
    GONetConvNeXtLike(num_blocks=2, name="GONetConvNeXtLike_2_blocs"),
    GONetShuffleNet(num_blocks=4, name="GONetShuffleNet_4_blocs"),
    GONetCrossLayer(name="GONetCrossLayer"),
]
epochs = 50
batch = 128

# Boucle d'entraînement pour chaque architecture
for model in instances:
    print(f"Initialisation : {model.name}")
    model.set_backbone()
    model.set_model()
    print(f"Entraînement : {model.name}")
    model.train_model(epochs=epochs, batch=batch)
    print(f"Terminé : {model.name}\n")

Affichons également l'évolution de l'accuracy pour les cinq architectures.

In [ ]:
# Crée un graphique pour toutes les courbes de policy_acc
plt.figure(figsize=(10, 6))

# Parcours de chaque instance
for model in instances:
    if hasattr(model, "policy_acc"):
        plt.plot(model.policy_acc, label=model.name)
    else:
        print(f"L'attribut 'policy_acc' est manquant pour : {model.name}")

# Mise en forme du graphique
plt.title("Évolution de la Policy Accuracy par architecture")
plt.xlabel("Époque")
plt.ylabel("Policy Accuracy")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

Enfin, entraînons le meilleur modèle : `GONetShuffleNet_4_blocs` sur 100 époques.

In [ ]:
instance_finale = GONetShuffleNet(
    num_blocks=4, name="GONetShuffleNet_4_blocs_final"
)
instance_finale.workflow(epochs=100, save_model=True)

In [ ]:
# Vérification et tracé de l'accuracy
if hasattr(instance_finale, "policy_acc"):
    acc = instance_finale.policy_acc
    epochs = list(range(1, len(acc) + 1))

    plt.figure(figsize=(8, 5))
    plt.plot(epochs, acc, label=instance_finale.name)

    # Affiche la valeur finale
    final_val = acc[-1] * 100
    plt.text(
        epochs[-1],
        acc[-1],
        f"{final_val:.2f}%",
        fontsize=10,
        verticalalignment="bottom",
        horizontalalignment="right",
    )

    # Mise en forme
    plt.title(f"Évolution de la Policy Accuracy – {instance_finale.name}")
    plt.xlabel("Époque")
    plt.ylabel("Policy Accuracy")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

else:
    print("L'instance ne contient pas d'attribut 'policy_acc'.")

Le modèle retenu est le modèle basé sur l'architecture ShuffleNet avec 4 blocs entraîné sur 100 époques avec une accuracy de 30,74%. 